In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [126]:
df = pd.read_csv(r"/content/drive/MyDrive/Deeplearning, Keras and Tensorflow/gradient_descent_insurance_data.csv", encoding="utf-8")

df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [ ]:
df.columns

Index(['age ', 'affordability', 'bought_insurance'], dtype='object')

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['age', 'affordability']], df.bought_insurance, test_size=0.2, random_state=25)

In [ ]:
len(X_train)

23

In [ ]:
X_train_scaled = X_train.copy()
X_train_scaled['age '] = X_train_scaled['age '] / 100

In [ ]:
X_test_scaled = X_test.copy()
X_test_scaled['age '] = X_test_scaled['age '] / 100

In [ ]:
from keras.optimizer import Adam

custom_lr = 0.001
custom_optimizer = Adam(learning_rate=lr)

model = keras.Sequential([
    keras.layers.Dense(1, input_shape=(2, ), activation='sigmoid', kernel_initializer='ones', bias_initializer='zeros')
])

model.compile(optimizer=custom_optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train_scaled, y_train, epochs = 1000)

Epoch 1/1000
1/1 [==============================] - 0s 403ms/step - loss: 0.7530 - accuracy: 0.6087
Epoch 2/1000
1/1 [==============================] - 0s 10ms/step - loss: 0.7526 - accuracy: 0.6087
Epoch 3/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.7523 - accuracy: 0.6087
Epoch 4/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.7520 - accuracy: 0.6087
Epoch 5/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.7517 - accuracy: 0.6087
Epoch 6/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.7514 - accuracy: 0.6087
Epoch 7/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.7511 - accuracy: 0.6087
Epoch 8/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.7508 - accuracy: 0.6087
Epoch 9/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.7505 - accuracy: 0.6087
Epoch 10/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.7502 - accuracy: 0.6087
Epoch 11/1000
1/

In [ ]:
model.evaluate(X_test_scaled, y_test)

1/1 [==============================] - 0s 190ms/step - loss: 0.6945 - accuracy: 0.3333


[0.694514274597168, 0.3333333432674408]

In [ ]:
X_test_scaled

,age,affordability
19,0.61,0
11,0.26,1
2,0.47,0
9,0.24,0
24,0.55,0
27,0.88,0


In [ ]:
y_test

19    1
11    1
2     0
9     0
24    0
27    0
Name: bought_insurance, dtype: int64

In [ ]:
model.predict(X_test_scaled)

1/1 [==============================] - 0s 47ms/step


array([[0.54423237],
       [0.6211227 ],
       [0.53032875],
       [0.5073945 ],
       [0.53828084],
       [0.5708346 ]], dtype=float32)

In [ ]:
coef,intercept = model.get_weights()

In [ ]:
coef

array([[0.3994953 ],
       [0.45674643]], dtype=float32)

In [ ]:
intercept

array([-0.0662988], dtype=float32)

Trying gradient descent practically

In [ ]:
import math

def sigmoid(x):
  return 1 / (1 + math.exp(-x))

In [ ]:
def prediction(age, affordability):
  weighted_sum = coef[0] * age + coef[1] * affordability + intercept
  return sigmoid(weighted_sum)

In [ ]:
prediction(0.61, 0)

0.5442324039660659

Implement gradient descent function from scratch

In [ ]:
# creating logg loss function for gradient descent

def log_loss_function(y_true, y_predicted):
  epsilon = 1e-15
  y_predicted_new = [max(i, epsilon) for i in y_predicted]
  y_predicted_new = [min(i, 1-epsilon) for i in y_predicted_new]
  y_predicted_new = np.array(y_predicted_new)
  return -np.mean(y_true * np.log(y_predicted_new) + (1-y_true) * np.log(1-y_predicted_new))


In [ ]:
def sigmoid_numpy(x_array):
  return 1 / (1 + np.exp(-x_array))

In [ ]:
# this func will find the weights and bias where the cost function is very minimum

# loss threshold value is taken from the tensorflow's last loss value (output of cell 78)

def gradient_descent(age, affordability, y_true, epochs, loss_threshold):
  w1 = w2 = 1
  bias = 0
  learning_rate = 0.5
  n = len(age) # or len(affordability) or len(y_true)

  for i in range(epochs):
    weighted_sum = w1 * age + w2 * affordability + bias
    y_predicted = sigmoid_numpy(weighted_sum)

    loss = log_loss_function(y_true, y_predicted)

    w1d = (1/n) * np.dot(np.transpose(age), (y_predicted - y_true))
    w2d = (1/n) * np.dot(np.transpose(affordability), (y_predicted - y_true))

    bias_d = np.mean(y_predicted-y_true)

    w1 = w1 - learning_rate * w1d
    w2 = w2 - learning_rate * w2d

    bias = bias - learning_rate * bias_d

    print(f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{round(loss,4)}')

    if round(loss,4) == loss_threshold:
      break

  return w1, w2, bias

In [ ]:
gradient_descent(X_train_scaled['age '], X_train_scaled['affordability'], y_train, 10000, 0.6827)
# print(X_train_scaled['age '])
# print(type(X_train_scaled['age ']))
# print(type(X_train_scaled['age '][0]))

Epoch:0, w1:0.9575783189401536, w2:0.945938810631127, bias:-0.05949213655752563, loss:0.753
Epoch:1, w1:0.9198877681094964, w2:0.8978116196249517, bias:-0.10892335672142978, loss:0.7375
Epoch:2, w1:0.8864753846028313, w2:0.8551799143758225, bias:-0.14930572848107992, loss:0.7261
Epoch:3, w1:0.8568596142497513, w2:0.8175264791646417, bias:-0.18170000732948313, loss:0.7176
Epoch:4, w1:0.8305599992156112, w2:0.7843015388037887, bias:-0.20715325783556515, loss:0.7114
Epoch:5, w1:0.8071188753634114, w2:0.7549589261389517, bias:-0.22665453340582814, loss:0.7068
Epoch:6, w1:0.7861151785373833, w2:0.7289811062880927, bias:-0.24110771444481224, loss:0.7035
Epoch:7, w1:0.767171631047171, w2:0.705894210332538, bias:-0.25131833335436715, loss:0.701
Epoch:8, w1:0.7499569913625792, w2:0.6852752455957956, bias:-0.25799056695280587, loss:0.6992
Epoch:9, w1:0.7341849785763339, w2:0.6667537787983452, bias:-0.26173089552723017, loss:0.6977
Epoch:10, w1:0.7196111883353804, w2:0.6500100679668924, bias:-0.2

(0.42544948830821455, 0.40768845711922846, -0.050963725974587634)

In [ ]:
# NOTE: stop the epochs when the loss is changing by very minimum value

In [ ]:
coef, intercept

(array([[0.3994953 ],
        [0.45674643]], dtype=float32),
 array([-0.0662988], dtype=float32))

In [ ]:
w1 = 0.3994953, w2 = 0.45674643, bias = 0.0662988